# Initial Set-up:
This section installs and imports any required libraries that will be used in the code below:

In [1]:
!pip install opacus
!pip install apricot-select numpy scikit-learn
!pip install kmedoids

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.4/254.4 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 13.0 MB/s eta 0:00:00
  Created wheel for apricot-select: filename=apricot_select-0.6.1-py3-none-any.whl size=48767 sha256=9d7f62ba1c0441e1bcf1f718bc27dd65b20eb781304e6e2e5b3f578cf80b4cd6
  Stored in directory: /root/.cache/pip/wheels/19/ce/18/f10e7debb348bf14d4ab90c8b657a91a79f1106b699a1121c3
Successfully built apricot-select
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.3/476.3 kB 33.4 MB/s eta 0:00:00


In [2]:
# For the CNN + DP-SGD
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset
import numpy as np
from opacus import PrivacyEngine
import time
import matplotlib.pyplot as plt

#For Lazy Greedy
from sklearn.metrics import pairwise_distances
import heapq
from sklearn.metrics import pairwise_distances
from apricot import FacilityLocationSelection
import time

#For K-medoids
import kmedoids
from sklearn.metrics.pairwise import euclidean_distances

# Select Dataset
All dataset arguments default to MNIST. This goes for the `run_experiment` function later. It is important to run `select_dataset` at least once as this loads the data. If you opt to run all cells, it will load MNIST by default.

In [14]:
#def select_dataset_greedy(name='MNIST', n_samples=100):
def select_dataset_greedy(name, n_samples):
  if name == "MNIST":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),])

    full_train = datasets.MNIST(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 5000
    val_size = 1000
    #val_size = len(full_train) - train_size
    # train_dataset, val_dataset = random_split(full_train, [train_size, val_size])
    train_dataset, subset_rest = random_split(full_train, [train_size, len(full_train) - train_size])
    val_dataset, _ = random_split(subset_rest, [val_size, len(subset_rest) - val_size])

    test_dataset = datasets.MNIST(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  elif name == "CIFAR":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])


    full_train = datasets.CIFAR10(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 40000
    val_size = len(full_train) - train_size
    #train_size = 5000
    #val_size = 1000
    # train_dataset, val_dataset = random_split(full_train, [train_size, val_size])
    train_dataset, subset_rest = random_split(full_train, [train_size, len(full_train) - train_size])
    val_dataset, _ = random_split(subset_rest, [val_size, len(subset_rest) - val_size])


    test_dataset = datasets.CIFAR10(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  train_data_array = train_dataset.dataset.data[train_dataset.indices]

  #----------------------MADI ADDED BECAUSE RAM SUCKS---------------------------

  #Create a smaller subset for FacilityLocationSelection to prevent crashes due to large memory usage
  #Adjusting the subset size for testing, e.g., 1000 samples
  subset_size_for_selection = min(1000, train_data_array.shape[0])
  # Randomly sample indices for the subset
  np.random.seed(SEED) # Ensure reproducibility
  random_indices = np.random.choice(train_data_array.shape[0], subset_size_for_selection, replace=False)
  subset_for_selection = train_data_array[random_indices]

  #-----------------------------------------------------------------------------

  selector = FacilityLocationSelection(n_samples=n_samples, metric='euclidean', optimizer='lazy', verbose=True)

  # Reshape the subset for selection
  subset_for_selection = subset_for_selection.reshape(subset_for_selection.shape[0], -1)
  #train_data_array = train_data_array.reshape(train_data_array.shape[0], -1)

  # Use the subset for selection
  selector.fit(subset_for_selection)
  #selector.fit(train_data_array)

  selected_indices = selector.ranking

  selected_train_dataset = Subset(train_dataset, selected_indices)

  train_loader = DataLoader(selected_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  val_loader   = DataLoader(val_dataset, batch_size=256, shuffle=False)
  test_loader  = DataLoader(test_dataset, batch_size=256, shuffle=False)

  return train_loader, val_loader, test_loader

In [3]:
#def select_dataset_kmedoids(name='MNIST', n_clusters=300):
def select_dataset_kmedoids(name, n_clusters):
  if name == "MNIST":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),])

    full_train = datasets.MNIST(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 50000
    val_size = len(full_train) - train_size
    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])

    test_dataset = datasets.MNIST(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  elif name == "CIFAR":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])


    full_train = datasets.CIFAR10(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 40000
    val_size = len(full_train) - train_size
    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])


    test_dataset = datasets.CIFAR10(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  train_data_array = train_dataset.dataset.data[train_dataset.indices]
  # Convert to numpy array first if it's a tensor, then convert type
  if isinstance(train_data_array, torch.Tensor):
    train_data_array = train_data_array.cpu().numpy()
  train_data_array = train_data_array.astype('float32')
  train_data_flat = train_data_array.reshape(train_data_array.shape[0], -1)

  # Create a smaller subset for kmedoids to prevent crashes due to large memory usage
  subset_size_for_kmedoids = min(5000, train_data_array.shape[0]) # Adjusted subset size
  np.random.seed(SEED) # Ensure reproducibility
  random_indices_kmedoids = np.random.choice(train_data_array.shape[0], subset_size_for_kmedoids, replace=False)
  subset_for_kmedoids = train_data_flat[random_indices_kmedoids]

  # Calculate distances only for the subset
  diss = euclidean_distances(subset_for_kmedoids)

  kmeds = kmedoids.KMedoids(n_clusters=n_clusters, method='fasterpam', random_state=0, metric='precomputed')
  model = kmeds.fit(diss)

  # Map the medoid indices from the subset back to the original dataset indices
  selected_subset_indices = model.medoid_indices_
  selected_original_indices = random_indices_kmedoids[selected_subset_indices]
  select = [train_dataset.indices[i] for i in selected_original_indices]

  selected_train_dataset = Subset(full_train, select)

  train_loader = DataLoader(selected_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  val_loader   = DataLoader(val_dataset, batch_size=256, shuffle=False)
  test_loader  = DataLoader(test_dataset, batch_size=256, shuffle=False)

  return train_loader, val_loader, test_loader

# Baseline CNN Model

This model is chosen in `run_experiment` if an invalid dataset is selected. I should probably replace this with a `ThrowException` instead though.

In [4]:
# baseline model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            MNIST Settings
        #=======================================#

        # self.conv1 = nn.Conv2d(1, 16, 3, 1)
        # self.conv2 = nn.Conv2d(16, 32, 3, 1)
        # self.fc1 = nn.Linear(32*12*12, 64)
        # self.fc2 = nn.Linear(64, 10)

        #=======================================#
        #            CIFAR-10 Settings
        #=======================================#

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=576, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        # x = F.relu(self.conv1(x))
        # x = F.relu(self.conv2(x))
        # x = F.max_pool2d(x, 2)
        # x = torch.flatten(x, 1)
        # x = F.relu(self.fc1(x))
        # return self.fc2(x)

        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## Model Architectures
MNIST and CIFAR-10 require different NN structures, so they have their own separate classes that get initialized in `run_experiment`.

In [5]:
# MNIST Model
class CNN_MNIST(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            MNIST Settings
        #=======================================#

        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32*12*12, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


# CIFAR-10 Model

class CNN_CIFAR10(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            CIFAR-10 Settings
        #=======================================#

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=576, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):

        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Model Training
Courtesy of Megha

In [6]:
def train_one_epoch(model, loader, optimizer):
    # train loop
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [7]:
@torch.no_grad()
def evaluate(model, loader):
    # evaluation
    model.eval()
    loss, correct = 0, 0
    for data, target in loader:
        data, target = data.to(device), target.to(device)
        out = model(data)
        loss += F.cross_entropy(out, target, reduction="sum").item()
        pred = out.argmax(1)
        correct += pred.eq(target).sum().item()
    loss /= len(loader.dataset)
    acc = 100. * correct / len(loader.dataset)
    return loss, acc

In [8]:
"""
dataset: either MNIST or CIFAR
I set dataset_name as a hyperparameter we can use for convenience and easy swapping
Otherwise, it defaults to MNIST, and invalid datasets give the base CNN model
This makes the dataset selection a modular component, should we want to choose other datasets
"""

def run_experiment(dataset='MNIST', use_dp=False, fixed_privacy_budget=False):

    # setup model and optimizer
    if dataset == 'MNIST':
      model = CNN_MNIST().to(device)
    elif dataset == "CIFAR":
      model = CNN_CIFAR10().to(device)

    # Defaults to base CNN class if no dataset is specified
    else:
      model = CNN().to(device)

    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

    privacy_engine = None
    if use_dp:

        if fixed_privacy_budget:

            privacy_engine = PrivacyEngine()

            # Calculates sigma based on target epsilon and delta
            model, optimizer, train_loader_dp = privacy_engine.make_private_with_epsilon(
              module=model,
              optimizer=optimizer,
              data_loader=train_loader,
              max_grad_norm=MAX_GRAD_NORM,
              target_delta=TARGET_DELTA,
              target_epsilon=TARGET_EPSILON,
              epochs=NUM_EPOCHS
        )
        else:

            privacy_engine = PrivacyEngine()

            # Otherwise takes sigma as a hyperparameter
            model, optimizer, train_loader_dp = privacy_engine.make_private(
              module=model,
              optimizer=optimizer,
              data_loader=train_loader,
              max_grad_norm=MAX_GRAD_NORM,
              noise_multiplier=NOISE_MULTIPLIER,
        )
    else:
        train_loader_dp = train_loader


    # train
    best_val_acc = 0
    epochs_no_improve = 0
    best_model_path = f"best_model{'_dp' if use_dp else ''}.pt"

    start_time = time.time()

    for epoch in range(1, EPOCHS + 1):
        train_loss = train_one_epoch(model, train_loader_dp, optimizer)
        val_loss, val_acc = evaluate(model, val_loader)

        eps = privacy_engine.get_epsilon(DELTA)

        print(f"[{'DP-SGD' if use_dp else 'Standard SGD'}] Epoch {epoch}: "
            + f"train_loss={train_loss:.4f}, "
            + f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}%"
            + f", ε={eps:.4f}" if use_dp else "")

        # if fixed_privacy_budget:
        #   eps = privacy_engine.get_epsilon(TARGET_DELTA)
        #   print(f", ε={eps:.4f}")

        # elif use_dp:
        #   eps = privacy_engine.get_epsilon(DELTA)
        #   print()

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
            torch.save(model.state_dict(), best_model_path)
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= PATIENCE:
            print(f"\nEarly stopping at epoch {epoch} (no improvement for {PATIENCE} epochs).")
            break


    end_time = time.time()
    total_time = end_time - start_time

    # test on best model
    model.load_state_dict(torch.load(best_model_path))
    test_loss, test_acc = evaluate(model, test_loader)
    final_eps = privacy_engine.get_epsilon(DELTA) if use_dp else None

    print(f"\n[{ 'DP-SGD' if use_dp else 'Standard-SGD'}]")
    print(f"Best val acc: {best_val_acc:.4f}%, Test acc: {test_acc:.4f}%\n" + (f" ε={final_eps:.4f}\n" if use_dp else "") + (f"Total runtime: {total_time:.4f} seconds"))
    return best_val_acc, test_acc, final_eps

# Initial Benchmarks
## Model Hyperparameters

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 128
LR = 0.01
EPOCHS = 20
SEED = 42
MAX_GRAD_NORM = 1.0
DELTA = 1e-5
PATIENCE = 5

# For dynamically calculated epsilon
NOISE_MULTIPLIER = 1.0

# For a fixed privacy budget (eps, del)
TARGET_EPSILON = 8
TARGET_DELTA = 1e-5
NUM_EPOCHS = 20

torch.manual_seed(SEED)
np.random.seed(SEED)

# TESTS:

## MNIST

### Lazy Greedy

In [ ]:
DATASET_NAME = "MNIST"
N_SAMPLES = 100
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

100%|██████████| 100/100 [00:00<00:00, 191it/s] 
/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/prv/prvs.py:67: RuntimeWarning: divide by zero encountered in log
  z = np.log(np.where(t > np.log(1 - q), (np.exp(t) + q - 1) / q, 1))
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/prv/prvs.py:70: RuntimeWarning: divide by zero encountered in log
  t > np.log(1 - q),



Running DP-SGD baseline for fixed (8, 1e-05)-DP


sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3196, val_loss=2.3156, val_acc=7.3000%, ε=1.4451
[DP-SGD] Epoch 2: train_loss=2.3186, val_loss=2.3144, val_acc=7.5000%, ε=2.1200
[DP-SGD] Epoch 3: train_loss=2.3167, val_loss=2.3126, val_acc=8.8000%, ε=2.6597
[DP-SGD] Epoch 4: train_loss=2.3137, val_loss=2.3102, val_acc=10.2000%, ε=3.1286
[DP-SGD] Epoch 5: train_loss=2.3100, val_loss=2.3070, val_acc=11.4000%, ε=3.5518
[DP-SGD] Epoch 6: train_loss=2.3054, val_loss=2.3036, val_acc=13.0000%, ε=3.9423
[DP-SGD] Epoch 7: train_loss=2.3003, val_loss=2.3001, val_acc=14.3000%, ε=4.3079
[DP-SGD] Epoch 8: train_loss=2.2946, val_loss=2.2963, val_acc=14.6000%, ε=4.6538
[DP-SGD] Epoch 9: train_loss=2.2885, val_loss=2.2925, val_acc=16.3000%, ε=4.9834
[DP-SGD] Epoch 10: train_loss=2.2824, val_loss=2.2890, val_acc=17.1000%, ε=5.2994
[DP-SGD] Epoch 11: train_loss=2.2767, val_loss=2.2854, val_acc=17.7000%, ε=5.6037
[DP-SGD] Epoch 12: train_loss=2.2705, val_loss=2.2815, val_acc=18.1000%, ε=5.8979
[DP-SGD] Epoch 13: train_los

(25.7, 25.94, np.float64(7.997143272367276))

In [ ]:
DATASET_NAME = "MNIST"
N_SAMPLES = 200
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

100%|██████████| 200/200 [00:00<00:00, 325it/s]
/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(



Running DP-SGD baseline for fixed (8, 1e-05)-DP


/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3110, val_loss=2.3097, val_acc=9.6000%, ε=1.7625
[DP-SGD] Epoch 2: train_loss=2.2955, val_loss=2.3061, val_acc=9.5000%, ε=2.4240
[DP-SGD] Epoch 3: train_loss=2.2905, val_loss=2.3017, val_acc=9.5000%, ε=2.9456
[DP-SGD] Epoch 4: train_loss=2.2895, val_loss=2.2969, val_acc=9.9000%, ε=3.3954
[DP-SGD] Epoch 5: train_loss=2.2904, val_loss=2.2912, val_acc=10.2000%, ε=3.7996
[DP-SGD] Epoch 6: train_loss=2.2807, val_loss=2.2849, val_acc=10.4000%, ε=4.1714
[DP-SGD] Epoch 7: train_loss=2.2776, val_loss=2.2783, val_acc=11.5000%, ε=4.5186
[DP-SGD] Epoch 8: train_loss=2.2664, val_loss=2.2709, val_acc=14.0000%, ε=4.8463
[DP-SGD] Epoch 9: train_loss=2.2590, val_loss=2.2630, val_acc=15.5000%, ε=5.1581
[DP-SGD] Epoch 10: train_loss=2.2405, val_loss=2.2540, val_acc=16.8000%, ε=5.4565
[DP-SGD] Epoch 11: train_loss=2.2374, val_loss=2.2444, val_acc=18.4000%, ε=5.7435
[DP-SGD] Epoch 12: train_loss=2.2177, val_loss=2.2340, val_acc=19.1000%, ε=6.0207
[DP-SGD] Epoch 13: train_loss

(32.0, 31.59, np.float64(7.990183308944778))

In [ ]:
DATASET_NAME = "MNIST"
N_SAMPLES = 300
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

100%|██████████| 300/300 [00:00<00:00, 429it/s]
/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(



Running DP-SGD baseline for fixed (8, 1e-05)-DP


sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3145, val_loss=2.3070, val_acc=9.0000%, ε=1.9068
[DP-SGD] Epoch 2: train_loss=2.3010, val_loss=2.3017, val_acc=9.4000%, ε=2.5514
[DP-SGD] Epoch 3: train_loss=2.2920, val_loss=2.2954, val_acc=11.9000%, ε=3.0612
[DP-SGD] Epoch 4: train_loss=2.2875, val_loss=2.2889, val_acc=14.4000%, ε=3.5012
[DP-SGD] Epoch 5: train_loss=2.2682, val_loss=2.2822, val_acc=17.7000%, ε=3.8968
[DP-SGD] Epoch 6: train_loss=2.2603, val_loss=2.2745, val_acc=17.2000%, ε=4.2607
[DP-SGD] Epoch 7: train_loss=2.2438, val_loss=2.2660, val_acc=16.4000%, ε=4.6005
[DP-SGD] Epoch 8: train_loss=2.2267, val_loss=2.2560, val_acc=15.7000%, ε=4.9213
[DP-SGD] Epoch 9: train_loss=2.2106, val_loss=2.2428, val_acc=17.7000%, ε=5.2264
[DP-SGD] Epoch 10: train_loss=2.1731, val_loss=2.2284, val_acc=18.4000%, ε=5.5184
[DP-SGD] Epoch 11: train_loss=2.1610, val_loss=2.2128, val_acc=18.7000%, ε=5.7993
[DP-SGD] Epoch 12: train_loss=2.1325, val_loss=2.1963, val_acc=18.7000%, ε=6.0705
[DP-SGD] Epoch 13: train_lo

(44.3, 45.03, np.float64(7.996436921675102))

### K-Medoids

In [10]:
DATASET_NAME = "MNIST"
N_CLUSTERS = 100
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.53MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 131kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.73MB/s]



Running DP-SGD baseline for fixed (8, 1e-05)-DP


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/prv/prvs.py:67: RuntimeWarning: divide by zero encountered in log
  z = np.log(np.where(t > np.log(1 - q), (np.exp(t) + q - 1) / q, 1))
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/prv/prvs.py:70: RuntimeWarning: divide by zero encountered in log
  t > np.log(1 - q),
sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to modul

[DP-SGD] Epoch 1: train_loss=2.3022, val_loss=2.3034, val_acc=8.6700%, ε=1.4451
[DP-SGD] Epoch 2: train_loss=2.3014, val_loss=2.3018, val_acc=9.0300%, ε=2.1200
[DP-SGD] Epoch 3: train_loss=2.2994, val_loss=2.2995, val_acc=9.5700%, ε=2.6597
[DP-SGD] Epoch 4: train_loss=2.2961, val_loss=2.2965, val_acc=10.2900%, ε=3.1286
[DP-SGD] Epoch 5: train_loss=2.2920, val_loss=2.2928, val_acc=11.0200%, ε=3.5518
[DP-SGD] Epoch 6: train_loss=2.2870, val_loss=2.2884, val_acc=12.1600%, ε=3.9423
[DP-SGD] Epoch 7: train_loss=2.2811, val_loss=2.2837, val_acc=14.1900%, ε=4.3079
[DP-SGD] Epoch 8: train_loss=2.2746, val_loss=2.2783, val_acc=16.9900%, ε=4.6538
[DP-SGD] Epoch 9: train_loss=2.2672, val_loss=2.2723, val_acc=19.5100%, ε=4.9834
[DP-SGD] Epoch 10: train_loss=2.2590, val_loss=2.2661, val_acc=21.8100%, ε=5.2994
[DP-SGD] Epoch 11: train_loss=2.2506, val_loss=2.2596, val_acc=24.0900%, ε=5.6037
[DP-SGD] Epoch 12: train_loss=2.2419, val_loss=2.2532, val_acc=24.8600%, ε=5.8979
[DP-SGD] Epoch 13: train_los

(29.3, 29.32, np.float64(7.997143272367274))

In [11]:
DATASET_NAME = "MNIST"
N_CLUSTERS = 200
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)


Running DP-SGD baseline for fixed (8, 1e-05)-DP


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3085, val_loss=2.2995, val_acc=10.8900%, ε=1.7625
[DP-SGD] Epoch 2: train_loss=2.3036, val_loss=2.2975, val_acc=10.7200%, ε=2.4240
[DP-SGD] Epoch 3: train_loss=2.2924, val_loss=2.2953, val_acc=10.9200%, ε=2.9456
[DP-SGD] Epoch 4: train_loss=2.2881, val_loss=2.2925, val_acc=11.2900%, ε=3.3954
[DP-SGD] Epoch 5: train_loss=2.2889, val_loss=2.2896, val_acc=11.7100%, ε=3.7996
[DP-SGD] Epoch 6: train_loss=2.2885, val_loss=2.2865, val_acc=11.3700%, ε=4.1714
[DP-SGD] Epoch 7: train_loss=2.2775, val_loss=2.2835, val_acc=10.9300%, ε=4.5186
[DP-SGD] Epoch 8: train_loss=2.2666, val_loss=2.2804, val_acc=10.8700%, ε=4.8463
[DP-SGD] Epoch 9: train_loss=2.2773, val_loss=2.2768, val_acc=11.1700%, ε=5.1581
[DP-SGD] Epoch 10: train_loss=2.2641, val_loss=2.2725, val_acc=12.8300%, ε=5.4565
[DP-SGD] Epoch 11: train_loss=2.2537, val_loss=2.2679, val_acc=15.7000%, ε=5.7435
[DP-SGD] Epoch 12: train_loss=2.2482, val_loss=2.2626, val_acc=17.3200%, ε=6.0207
[DP-SGD] Epoch 13: train_

(17.32, 17.92, np.float64(7.293493928359672))

In [12]:
DATASET_NAME = "MNIST"
N_CLUSTERS = 300
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)


Running DP-SGD baseline for fixed (8, 1e-05)-DP


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.2987, val_loss=2.3066, val_acc=9.6000%, ε=1.9068
[DP-SGD] Epoch 2: train_loss=2.2924, val_loss=2.2981, val_acc=10.8900%, ε=2.5514
[DP-SGD] Epoch 3: train_loss=2.2893, val_loss=2.2874, val_acc=12.9600%, ε=3.0612
[DP-SGD] Epoch 4: train_loss=2.2746, val_loss=2.2752, val_acc=15.7600%, ε=3.5012
[DP-SGD] Epoch 5: train_loss=2.2601, val_loss=2.2614, val_acc=18.9700%, ε=3.8968
[DP-SGD] Epoch 6: train_loss=2.2454, val_loss=2.2457, val_acc=22.2600%, ε=4.2607
[DP-SGD] Epoch 7: train_loss=2.2162, val_loss=2.2265, val_acc=26.0800%, ε=4.6005
[DP-SGD] Epoch 8: train_loss=2.1883, val_loss=2.2052, val_acc=29.4300%, ε=4.9213
[DP-SGD] Epoch 9: train_loss=2.1875, val_loss=2.1811, val_acc=33.7600%, ε=5.2264
[DP-SGD] Epoch 10: train_loss=2.1396, val_loss=2.1519, val_acc=38.4600%, ε=5.5184
[DP-SGD] Epoch 11: train_loss=2.1240, val_loss=2.1206, val_acc=42.1300%, ε=5.7993
[DP-SGD] Epoch 12: train_loss=2.0665, val_loss=2.0879, val_acc=45.9100%, ε=6.0705
[DP-SGD] Epoch 13: train_l

(65.16, 65.88, np.float64(7.9964369216751034))

## CIFAR-10

### Lazy Greedy

In [15]:
DATASET_NAME = "CIFAR"

N_SAMPLES = 50
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

100%|██████████| 170M/170M [00:13<00:00, 12.8MB/s]
100%|██████████| 50.0/50.0 [00:00<00:00, 1.12kit/s]
/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/prv/prvs.py:67: RuntimeWarning: divide by zero encountered in log
  z = np.log(np.where(t > np.log(1 - q), (np.exp(t) + q - 1) / q, 1))
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/prv/prvs.py:70: RuntimeWarning: divide by zero encountered in log
  t > np.log(1 -


Running DP-SGD baseline for fixed (8, 1e-05)-DP


sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3100, val_loss=2.3071, val_acc=10.5000%, ε=1.4451
[DP-SGD] Epoch 2: train_loss=2.3097, val_loss=2.3071, val_acc=10.4100%, ε=2.1200
[DP-SGD] Epoch 3: train_loss=2.3090, val_loss=2.3071, val_acc=10.4100%, ε=2.6597
[DP-SGD] Epoch 4: train_loss=2.3081, val_loss=2.3071, val_acc=10.3900%, ε=3.1286
[DP-SGD] Epoch 5: train_loss=2.3070, val_loss=2.3071, val_acc=10.1500%, ε=3.5518
[DP-SGD] Epoch 6: train_loss=2.3054, val_loss=2.3072, val_acc=9.9700%, ε=3.9423

Early stopping at epoch 6 (no improvement for 5 epochs).

[DP-SGD]
Best val acc: 10.5000%, Test acc: 10.9000%
 ε=3.9423
Total runtime: 14.7384 seconds


(10.5, 10.9, np.float64(3.942293752673243))

In [16]:
DATASET_NAME = "CIFAR"
N_SAMPLES = 100
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

100%|██████████| 100/100 [00:00<00:00, 1.64kit/s]
/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/prv/prvs.py:67: RuntimeWarning: divide by zero encountered in log
  z = np.log(np.where(t > np.log(1 - q), (np.exp(t) + q - 1) / q, 1))
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/prv/prvs.py:70: RuntimeWarning: divide by zero encountered in log
  t > np.log(1 - q),



Running DP-SGD baseline for fixed (8, 1e-05)-DP


sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3182, val_loss=2.3072, val_acc=10.0500%, ε=1.4451
[DP-SGD] Epoch 2: train_loss=2.3181, val_loss=2.3072, val_acc=10.0500%, ε=2.1200
[DP-SGD] Epoch 3: train_loss=2.3177, val_loss=2.3070, val_acc=10.0900%, ε=2.6597
[DP-SGD] Epoch 4: train_loss=2.3171, val_loss=2.3069, val_acc=10.0400%, ε=3.1286
[DP-SGD] Epoch 5: train_loss=2.3164, val_loss=2.3066, val_acc=9.9200%, ε=3.5518
[DP-SGD] Epoch 6: train_loss=2.3154, val_loss=2.3064, val_acc=9.9400%, ε=3.9423
[DP-SGD] Epoch 7: train_loss=2.3143, val_loss=2.3062, val_acc=9.8800%, ε=4.3079
[DP-SGD] Epoch 8: train_loss=2.3132, val_loss=2.3060, val_acc=9.9700%, ε=4.6538

Early stopping at epoch 8 (no improvement for 5 epochs).

[DP-SGD]
Best val acc: 10.0900%, Test acc: 10.3100%
 ε=4.6538
Total runtime: 19.9943 seconds


(10.09, 10.31, np.float64(4.65376554171542))

In [17]:
DATASET_NAME = "CIFAR"
N_SAMPLES = 200
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

100%|██████████| 200/200 [00:00<00:00, 1.96kit/s]
/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(



Running DP-SGD baseline for fixed (8, 1e-05)-DP


/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3042, val_loss=2.3034, val_acc=10.0700%, ε=1.7625
[DP-SGD] Epoch 2: train_loss=2.3084, val_loss=2.3032, val_acc=10.1200%, ε=2.4240
[DP-SGD] Epoch 3: train_loss=2.3065, val_loss=2.3030, val_acc=10.1600%, ε=2.9456
[DP-SGD] Epoch 4: train_loss=2.3103, val_loss=2.3027, val_acc=10.2100%, ε=3.3954
[DP-SGD] Epoch 5: train_loss=2.3032, val_loss=2.3025, val_acc=10.1400%, ε=3.7996
[DP-SGD] Epoch 6: train_loss=2.3003, val_loss=2.3022, val_acc=10.1800%, ε=4.1714
[DP-SGD] Epoch 7: train_loss=2.3043, val_loss=2.3020, val_acc=10.2900%, ε=4.5186
[DP-SGD] Epoch 8: train_loss=2.3025, val_loss=2.3017, val_acc=10.5900%, ε=4.8463
[DP-SGD] Epoch 9: train_loss=2.3016, val_loss=2.3014, val_acc=10.0100%, ε=5.1581
[DP-SGD] Epoch 10: train_loss=2.3009, val_loss=2.3011, val_acc=9.9000%, ε=5.4565
[DP-SGD] Epoch 11: train_loss=2.2939, val_loss=2.3008, val_acc=10.0400%, ε=5.7435
[DP-SGD] Epoch 12: train_loss=2.2939, val_loss=2.3005, val_acc=10.0300%, ε=6.0207
[DP-SGD] Epoch 13: train_l

(10.59, 10.64, np.float64(6.2891640598459615))

In [18]:
DATASET_NAME = "CIFAR"
N_SAMPLES = 300
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

100%|██████████| 300/300 [00:00<00:00, 1.77kit/s]
/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(



Running DP-SGD baseline for fixed (8, 1e-05)-DP


sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3072, val_loss=2.3050, val_acc=9.1000%, ε=1.9068
[DP-SGD] Epoch 2: train_loss=2.3081, val_loss=2.3047, val_acc=9.0200%, ε=2.5514
[DP-SGD] Epoch 3: train_loss=2.3062, val_loss=2.3044, val_acc=8.6700%, ε=3.0612
[DP-SGD] Epoch 4: train_loss=2.3040, val_loss=2.3039, val_acc=8.8900%, ε=3.5012
[DP-SGD] Epoch 5: train_loss=2.3053, val_loss=2.3033, val_acc=9.0600%, ε=3.8968
[DP-SGD] Epoch 6: train_loss=2.3000, val_loss=2.3028, val_acc=9.0600%, ε=4.2607

Early stopping at epoch 6 (no improvement for 5 epochs).

[DP-SGD]
Best val acc: 9.1000%, Test acc: 9.4000%
 ε=4.2607
Total runtime: 19.4887 seconds


(9.1, 9.4, np.float64(4.260672744324618))

In [23]:
DATASET_NAME = "CIFAR"
N_SAMPLES = 500
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

100%|██████████| 500/500 [00:00<00:00, 1.33kit/s]
/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(



Running DP-SGD baseline for fixed (8, 1e-05)-DP


sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3039, val_loss=2.3064, val_acc=9.7400%, ε=1.9909
[DP-SGD] Epoch 2: train_loss=2.3033, val_loss=2.3060, val_acc=9.6900%, ε=2.6240
[DP-SGD] Epoch 3: train_loss=2.3047, val_loss=2.3056, val_acc=9.7000%, ε=3.1260
[DP-SGD] Epoch 4: train_loss=2.2962, val_loss=2.3050, val_acc=9.8700%, ε=3.5598
[DP-SGD] Epoch 5: train_loss=2.2977, val_loss=2.3043, val_acc=10.1900%, ε=3.9499
[DP-SGD] Epoch 6: train_loss=2.3010, val_loss=2.3035, val_acc=11.2300%, ε=4.3089
[DP-SGD] Epoch 7: train_loss=2.2985, val_loss=2.3025, val_acc=13.3500%, ε=4.6443
[DP-SGD] Epoch 8: train_loss=2.2974, val_loss=2.3016, val_acc=15.7900%, ε=4.9609
[DP-SGD] Epoch 9: train_loss=2.2937, val_loss=2.3007, val_acc=16.0200%, ε=5.2620
[DP-SGD] Epoch 10: train_loss=2.2925, val_loss=2.2997, val_acc=15.6800%, ε=5.5503
[DP-SGD] Epoch 11: train_loss=2.2897, val_loss=2.2987, val_acc=15.2100%, ε=5.8275
[DP-SGD] Epoch 12: train_loss=2.2878, val_loss=2.2977, val_acc=14.6500%, ε=6.0951
[DP-SGD] Epoch 13: train_loss

(16.02, 15.52, np.float64(6.60613552545733))

### K-Medoids

In [19]:
DATASET_NAME = "CIFAR"
N_CLUSTERS = 100
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)


Running DP-SGD baseline for fixed (8, 1e-05)-DP


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/prv/prvs.py:67: RuntimeWarning: divide by zero encountered in log
  z = np.log(np.where(t > np.log(1 - q), (np.exp(t) + q - 1) / q, 1))
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/prv/prvs.py:70: RuntimeWarning: divide by zero encountered in log
  t > np.log(1 - q),
sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to modul

[DP-SGD] Epoch 1: train_loss=2.3057, val_loss=2.3064, val_acc=10.3100%, ε=1.4451
[DP-SGD] Epoch 2: train_loss=2.3051, val_loss=2.3064, val_acc=10.3200%, ε=2.1200
[DP-SGD] Epoch 3: train_loss=2.3040, val_loss=2.3064, val_acc=10.5400%, ε=2.6597
[DP-SGD] Epoch 4: train_loss=2.3024, val_loss=2.3064, val_acc=10.6000%, ε=3.1286
[DP-SGD] Epoch 5: train_loss=2.3004, val_loss=2.3064, val_acc=10.7700%, ε=3.5518
[DP-SGD] Epoch 6: train_loss=2.2981, val_loss=2.3064, val_acc=10.9500%, ε=3.9423
[DP-SGD] Epoch 7: train_loss=2.2955, val_loss=2.3064, val_acc=10.9500%, ε=4.3079
[DP-SGD] Epoch 8: train_loss=2.2927, val_loss=2.3064, val_acc=10.9000%, ε=4.6538
[DP-SGD] Epoch 9: train_loss=2.2894, val_loss=2.3065, val_acc=10.7700%, ε=4.9834
[DP-SGD] Epoch 10: train_loss=2.2858, val_loss=2.3065, val_acc=10.6300%, ε=5.2994
[DP-SGD] Epoch 11: train_loss=2.2821, val_loss=2.3067, val_acc=10.6000%, ε=5.6037

Early stopping at epoch 11 (no improvement for 5 epochs).

[DP-SGD]
Best val acc: 10.9500%, Test acc: 11.2

(10.95, 11.21, np.float64(5.603719670759895))

In [20]:
DATASET_NAME = "CIFAR"
N_CLUSTERS = 200
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)


Running DP-SGD baseline for fixed (8, 1e-05)-DP


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3177, val_loss=2.3053, val_acc=10.0400%, ε=1.7625
[DP-SGD] Epoch 2: train_loss=2.3148, val_loss=2.3053, val_acc=10.0500%, ε=2.4240
[DP-SGD] Epoch 3: train_loss=2.3132, val_loss=2.3052, val_acc=9.7200%, ε=2.9456
[DP-SGD] Epoch 4: train_loss=2.3092, val_loss=2.3052, val_acc=9.4900%, ε=3.3954
[DP-SGD] Epoch 5: train_loss=2.3116, val_loss=2.3051, val_acc=9.3500%, ε=3.7996
[DP-SGD] Epoch 6: train_loss=2.2991, val_loss=2.3051, val_acc=9.2400%, ε=4.1714
[DP-SGD] Epoch 7: train_loss=2.3049, val_loss=2.3052, val_acc=9.2200%, ε=4.5186

Early stopping at epoch 7 (no improvement for 5 epochs).

[DP-SGD]
Best val acc: 10.0500%, Test acc: 9.7000%
 ε=4.5186
Total runtime: 23.0196 seconds


(10.05, 9.7, np.float64(4.518568053726991))

In [21]:
DATASET_NAME = "CIFAR"
N_CLUSTERS = 300
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)


Running DP-SGD baseline for fixed (8, 1e-05)-DP


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3041, val_loss=2.3050, val_acc=9.8100%, ε=1.9068
[DP-SGD] Epoch 2: train_loss=2.3024, val_loss=2.3046, val_acc=9.8400%, ε=2.5514
[DP-SGD] Epoch 3: train_loss=2.3016, val_loss=2.3040, val_acc=10.0100%, ε=3.0612
[DP-SGD] Epoch 4: train_loss=2.3017, val_loss=2.3031, val_acc=10.1600%, ε=3.5012
[DP-SGD] Epoch 5: train_loss=2.2999, val_loss=2.3022, val_acc=10.3000%, ε=3.8968
[DP-SGD] Epoch 6: train_loss=2.2956, val_loss=2.3011, val_acc=10.5900%, ε=4.2607
[DP-SGD] Epoch 7: train_loss=2.2876, val_loss=2.3000, val_acc=10.8200%, ε=4.6005
[DP-SGD] Epoch 8: train_loss=2.2894, val_loss=2.2989, val_acc=10.9800%, ε=4.9213
[DP-SGD] Epoch 9: train_loss=2.2900, val_loss=2.2979, val_acc=11.3900%, ε=5.2264
[DP-SGD] Epoch 10: train_loss=2.2858, val_loss=2.2972, val_acc=11.5200%, ε=5.5184
[DP-SGD] Epoch 11: train_loss=2.2855, val_loss=2.2964, val_acc=11.5300%, ε=5.7993
[DP-SGD] Epoch 12: train_loss=2.2848, val_loss=2.2954, val_acc=11.6500%, ε=6.0705
[DP-SGD] Epoch 13: train_lo

(15.91, 16.87, np.float64(7.9964369216751034))

In [22]:
DATASET_NAME = "CIFAR"
N_CLUSTERS = 500
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)


Running DP-SGD baseline for fixed (8, 1e-05)-DP


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.


[DP-SGD] Epoch 1: train_loss=2.3029, val_loss=2.3045, val_acc=8.7100%, ε=1.9909
[DP-SGD] Epoch 2: train_loss=2.3055, val_loss=2.3038, val_acc=8.9000%, ε=2.6240
[DP-SGD] Epoch 3: train_loss=2.3010, val_loss=2.3029, val_acc=9.1100%, ε=3.1260
[DP-SGD] Epoch 4: train_loss=2.2999, val_loss=2.3019, val_acc=9.5100%, ε=3.5598
[DP-SGD] Epoch 5: train_loss=2.3006, val_loss=2.3008, val_acc=9.5900%, ε=3.9499
[DP-SGD] Epoch 6: train_loss=2.2954, val_loss=2.2997, val_acc=9.5900%, ε=4.3089
[DP-SGD] Epoch 7: train_loss=2.2926, val_loss=2.2987, val_acc=9.5900%, ε=4.6443
[DP-SGD] Epoch 8: train_loss=2.2938, val_loss=2.2976, val_acc=9.7100%, ε=4.9609
[DP-SGD] Epoch 9: train_loss=2.2878, val_loss=2.2965, val_acc=9.8600%, ε=5.2620
[DP-SGD] Epoch 10: train_loss=2.2820, val_loss=2.2954, val_acc=10.3700%, ε=5.5503
[DP-SGD] Epoch 11: train_loss=2.2816, val_loss=2.2943, val_acc=10.9600%, ε=5.8275
[DP-SGD] Epoch 12: train_loss=2.2781, val_loss=2.2929, val_acc=11.5300%, ε=6.0951
[DP-SGD] Epoch 13: train_loss=2.27

(13.51, 14.03, np.float64(7.995831917202194))